In [4]:
has_file = []
for pi in PrintIssue.objects.all():
    try:
        data = pi.get_page_text_content(2)
    except FileNotFoundError:
        pass
    else:
        has_file.append(pi)

In [60]:
import re
import pprint
positions = 'redaktør', 'desksjef', 'nyhetsleder', 'nettredaktør', 'fotosjef', 'debattredaktør', 'magasinredaktør'
pattern =  re.compile(r'^(?P<title>\w+):\s(?P<name>[-\w ]+)$', flags = re.M | re.I)
def find_positions(issue):
    content = issue.get_page_text_content(2,100)
    #print(content)
    matches = pattern.finditer(content)
    return {d['title']: d['name'] for d in [m.groupdict() for m in matches] if d['title'] in positions}

def add_to_stint(title, name, date):
    contributor = Contributor.objects.get(display_name=name)
    position, _ = Position.objects.get_or_create(title=title)
    defaults = {'start_date': date, 'end_date': date}
    stint, _ = Stint.objects.get_or_create(defaults=defaults, position=position, contributor=contributor)
    stint.end_date = max(stint.end_date, date)
    stint.start_date = min(stint.start_date, date)
    stint.save()
     
    
for issue in has_file:
    data = find_positions(issue)
    content = issue.get_page_text_content(2,100)
    print(issue)
    pprint.pprint(data)
    date = issue.issue.publication_date
    for title, name in data.items():
        add_to_stint(title, name, date)
    


/media/pdf/8f7b66a2e82a11e68c22f23c916e5154/universitas_2017-4.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/0aae96f6f8ab11e68c22f23c916e5154/universitas_2017-7.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/dc03a8b4d7e411e6b0dcf23c916e5154/universitas_2017-1.pdf
{'debattredaktør': 'Knut Arne Oseid',
 'desksjef': 'Lise Blekastad',
 'fotosjef': 'Adrian Nielsen',
 'magasinredaktør': 'Signe Rosenlund-Hauglid',
 'nettredaktør': 'Sondre Myhre',
 'nyhetsleder': 'Birk Tjeldflaat Helle',
 'redaktør': 'Torgeir Mortensen'}
/media/pdf/3d392c0edd2a11e6b0dcf23

In [17]:
Byline.objects.filter(story__story_type__section__title='Kultur', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Anmeldelser', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Nyhet', credit='by').update(credit='text')
Byline.objects.filter(story__story_type__section__title='Debatt', credit='text').update(credit='by')

0

In [23]:
def add_stints_from_bylines(title, credit, exclude=['Debatt']):
    """Add stints based on byline credits"""
    position, _ = Position.objects.get_or_create(title=title)
    for person in Contributor.objects.all():
        bylines = person.byline_set.filter(
            credit__contains=credit
        ).exclude( # unpublished
            story__publication_date=None
        ).exclude( # no opinion pieces
            story__story_type__section__title__in=exclude
        ).order_by(
            'story__publication_date'
        )
        if bylines.count() > 1:  # ignore people with just one byline
            first_byline = bylines.first().story.publication_date.date()
            last_byline = bylines.last().story.publication_date.date()
            stint, created = Stint.objects.get_or_create(
                dict(start_date=first_byline, end_date=last_byline),
                contributor=person, position=position)
            if not created:
                stint.start_date = min(stint.start_date, first_byline)
                stint.end_date = max(stint.end_date, last_byline)
                stint.save()
            print(f'{stint} {bylines.count()}')
       
    
add_stints_from_bylines('journalist', 'text')
add_stints_from_bylines('fotograf', 'photo')
add_stints_from_bylines('oversetter', 'translation')
add_stints_from_bylines('illustratør', 'illustration')

journalist Kirsty Johansen 9
journalist Hanna Skotheim 43
journalist Henrik Evertsson 7
journalist Hallvard Østtveit Barbogen 20
journalist Anders R. Christensen 245
journalist Kjetil Nordengen 45
journalist Ketil Blom 12
journalist Linda Johnsen Ørstavik 48
journalist Lise Grønskar 46
journalist Melike Leblebicioğlu 25
journalist Mathias Vedeler 52
journalist Cecilia Johansen Eriksen 4
journalist Ørjan Ryland 5
journalist Hans Dalane-Hval 14
journalist Thorbjørn Kringlebotn Borlaug 66
journalist Birte Nystad Magnussen 2
journalist Journalist I Universitas 9
journalist Magnus Lysberg 25
journalist Julie Nordby Egeland 28
journalist Eirik Omvik 87
journalist Anders Ballangrud 74
journalist Are W. Sandvik 66
journalist Emma Tollersrud 14
journalist Jonas Øren 7
journalist Maria Terese Kittilsen 37
journalist Caroline Rugeldal 63
journalist Torgeir Mortensen 105
journalist Benedicte Tobiassen 62
journalist Maria Reinertsen 64
journalist Mina Gerhardsen 2
journalist Mads Randen 40
journali

journalist Embret T. Rognerød 25
journalist Universitas´ Juleølpanel 3
journalist Line Steine Oma 4
journalist Simen Tallaksen 66
journalist Ulrikke Bjørnsdatter Aarseth 10
journalist Margrethe Løøv 27
journalist Marie De Rosa 70
journalist Øyvind Bosnes Engen 158
journalist Ingvild Jacobsen 38
journalist Amund Trellevik 30
journalist Vegard Tjørhom 15
journalist Anders Mathias Johansen 14
journalist Katharina Simonsen 12
journalist Emil Flatø 46
journalist Magnus Løvold 31
journalist Nordis Tennes 55
journalist Andreas Sveen Bjørnstad 18
journalist Eir Torvik 56
journalist Lise Marie Lunaas Holt 16
journalist Karoline Sanna 8
journalist Nareen Mizoory 7
journalist Dag Grytli 34
journalist Caroline Furru 4
journalist Cecilie Einarson Pérez 9
journalist Mari K. S. Gjelstad 12
journalist Universitas’ Sommerøltestpanel 2
journalist Martin Grevstad 14
journalist Marianne Aasen 2
journalist Marit M. Simonsen 6
journalist Tone Magni F. Vestheim 2
journalist Bendik Løve 52
journalist Mikael L

fotograf Mathilde Oda Meyer Jørgensen 2
fotograf Hanna Hjardar 15
fotograf Angelique Culvin 18
fotograf Hilde Raa 13
fotograf Hanne Marie Lenth Solbø 12
fotograf Marianne Demmo 3
fotograf Dorthe Karlsen 34
fotograf Lovise Steinkjellå 2
fotograf Eirik Bryhn Jacobsen 4
fotograf Sara Nordby 4
fotograf Bjørn Brekke 21
fotograf Berit Andreassen 10
fotograf Khai Nguyen 4
fotograf Åshild Støylen 195
fotograf Halvor Finess Tretvoll 4
fotograf Erik Møller Solheim 2
fotograf Mona Tandsether 12
fotograf Håkon Rasmus Bergseth 6
fotograf Marianne Johansen 24
fotograf Solveig Schanke Eikum 9
fotograf Sunniva Trengereid 8
fotograf Håkon Mosvold 83
fotograf Fredrik Arff 50
fotograf Håkon Mosvold Larsen 10
fotograf Audun Kolstad Wiig 2
fotograf Ellinor Bente Dalbye 109
fotograf Elisabeth Johansen 16
fotograf Svein Egil Hatlevik 2
fotograf Espen Gleditsch 2
fotograf Lise Falck 24
fotograf Ida Marie Jahr 2
fotograf Camilla Tully 13
fotograf Leif Martin Kirknes 11
fotograf Kristine Gramstad 24
fotograf Øy

In [24]:
from django.utils import timezone

def update_active(inactive_limit=timezone.timedelta(days=150)):
    today = timezone.now().date()
    for contributor in Contributor.objects.all():
        all_bylines = contributor.byline_set.exclude(story__publication_date=None)
        debatt_bylines = all_bylines.filter(story__story_type__section__title='Debatt')
        status = Contributor.UNKNOWN
        if all_bylines.count() == 0:
            print(contributor)
        elif all_bylines.count() == debatt_bylines.count():
            status = contributor.EXTERNAL
        elif contributor.stint_set.count():
            latest = contributor.stint_set.latest('end_date')
            if today - latest.end_date > inactive_limit:
                status = contributor.RETIRED
            else:
                status = contributor.ACTIVE
        if status != contributor.status:
            contributor.status=status 
            contributor.save(update_fields=['status'])
        
        print(f'{str(contributor):<40} {contributor.get_status_display():<10} {contributor.byline_set.count():>5}')

update_active()
unknowns = Contributor.objects.filter(status=Contributor.UNKNOWN)
active = Contributor.objects.filter(status=Contributor.ACTIVE)
retired = Contributor.objects.filter(status=Contributor.RETIRED)
print(unknowns.count(), active.count(), retired.count())

Ingrid Os                                Ekstern        1
Jan Frich                                Ekstern        4
Kirsty Johansen                          Slutta         9
Kristoffer Guttorm Camillo Pedersen      Ekstern        1
Morten Nordby                            Ekstern        1
Hanna Skotheim                           Aktiv         46
Henrik Evertsson                         Slutta       116
Hallvard Østtveit Barbogen               Slutta        23
Siri Halland Nesse                       Ekstern        1
Anders R. Christensen                    Slutta       252
Kjetil Nordengen                         Slutta        52
Silje Ramstad                            Slutta        77
Ketil Blom                               Slutta       467
Linda Johnsen Ørstavik                   Slutta        53
Lise Grønskar                            Slutta        51
Melike Leblebicioğlu                     Slutta        30
Mathias Vedeler                          Slutta        58
Cecilia Johans

Kjetil Strømme                           Slutta       129
Bjørn–Egil Mikalsen                      Slutta        69
Anders Waaler Kemp                       Slutta        50
Henning Reinton                          Slutta        58
Sylvia-Angélique Culvin                  Aktiv          2
Ingeborg Sleipnes Sivertsen              Ukjent         4
Xueqi Pang                               Aktiv          9
Gard Oterholm                            Aktiv         61
Julie Brundtland Løvseth                 Aktiv          8
Vilde Borse                              Aktiv          6
Indigo Trigg-Hauger                      Aktiv         23
Kenneth Haug                             Aktiv          8
Dorthe Karlsen Og Matthis Kleeb          Ukjent         1
Alexander John Smith-Hald                Ekstern        1
Steffen S. Aagedal                       Slutta        26
Emili Knutson                            Aktiv         12
Emilie Solberg                           Aktiv         14
Mathilde Oda M

Simen Ekern                              Ukjent         1
Fredrik Poulsen                          Ukjent         1
Tore Kristian Oland                      Ekstern        4
Høyres Studenter                         Ekstern        2
Lars Joakim Hanssen                      Ekstern        1
Marit K. Slotnæs                         Ekstern        1
Wencke Mühleisen                         Ekstern        1
Odd Eidem Tveit                          Ukjent         1
Jenny Westerstrand                       Ekstern        1
Åsa Eldén                                Ekstern        1
Hilde Haualand                           Ekstern        1
Kristoffer Kjølberg                      Ukjent         2
Bulldozer                                Ukjent         3
Eva Lundgren                             Ekstern        1
Øivind Støle                             Ekstern        2
Katinka Salomonsen Grane                 Ukjent         1
Ingrid K. Lund                           Slutta        12
Knarkus       

Lise Marie Danielsen                     Slutta         8
Ole Peder Giæver                         Slutta        83
Master Of Oslo                           Ukjent         1
Christine Viland                         Ekstern        1
Ida Waal                                 Ekstern        1
Svein Egil Hatlevik                      Slutta       143
Tiden                                    Ukjent         1
Espen Gleditsch                          Slutta         9
Sigurd Kihl                              Slutta        45
Sigurd K Berg                            Ekstern        9
Peter Glen Student                       Ekstern        1
Ingunn Helstrup                          Ekstern        2
Trond Larsen De Radikale                 Ekstern        2
Harriet Bjerrum Nielsen Kollegiemedlem   Ekstern        1
Ingvild Berg                             Slutta        58
Professor Kaare Norum                    Ekstern        1
For Megaparty                            Ekstern        1
Kjetil Moløkke

Bjørn Niklas Sjøstrøm                    Ekstern        1
Kjartan Lindøe                           Ekstern        2
På Vegne Av Sio                          Ekstern        1
Lars Raaum                               Ekstern        2
Anne Eriksen                             Ekstern        1
Tonje Eliasson                           Slutta        30
Lene Hoem Røysum                         Slutta        20
Håvar Austgard                           Ekstern        1
Mari Reitzel Bjerke                      Ekstern        3
Mjøsen Golfklubb                         Ekstern        1
Johan Brockfield                         Ekstern        1
Jarle M. Alvheim                         Ekstern        1
Danem                                    Ukjent         2
Baser                                    Ukjent         1
Ansjos                                   Ukjent         1
5. Etasje                                Ukjent         1
Trollkarlen                              Ukjent         1
Kim Sølve Mads

...

Representant I Kirke-                    Ekstern        1
Elin Nesje Vestli                        Ekstern        1
Maria Svendsen                           Ekstern        1
Aina Stenersen                           Ekstern        5
Helene Bjerke Fredheim                   Ekstern        1
Marit M. Simonsen                        Slutta         9
Stine Elise Christensen                  Ekstern        1
Tone Magni F. Vestheim                   Slutta         2
Helge Holgersen                          Ekstern        2
Ståle Pallesen                           Ekstern        2
Ketil Mathiassen                         Ekstern        2
Fanny Duckert                            Ekstern        2
Kjell Lars Berge                         Ekstern        2
Powerpuff                                Ukjent         1
Pål Buset                                Ekstern        1
Bendik Løve                              Slutta        61
Mikael Lunde                             Slutta        33
Lektorprogramm

Eirik Vatnøy                             Ekstern        1
Marius Doksheim                          Ekstern        1
Janne M. Log                             Ekstern        3
Nora Zomlot                              Ekstern        1
Hans J. Skjong                           Slutta       162
Rolf V. Olsen                            Ekstern        1
Lars Heltne                              Slutta        23
Bendik Baksaas                           Slutta        39
Mia Caroline Bratz                       Slutta        25
Håkon Frede Foss                         Slutta        67
Ingrid Jørgensen                         Slutta         5
Ingvild Sagmoen                          Slutta        47
Liv-Kristin Rød Korssjøen                Ekstern        6
Marius Næss Olsen                        Ekstern        1
Sander Brink                             Slutta         2
Vegard Røneid Erikstad                   Slutta        48
Kristian Wikborg Wiese                   Slutta        23
Sjalg Bøhmer V

Benjamin Oliver                          Ukjent         5
Petter Aaslestad                         Ekstern        2
Therese Eia Lerøen                       Ekstern       10
Himanshu Gulati                          Ekstern        3
Edle Ravndal                             Ekstern        2
Kristoffer Kaayne Kaalsaas               Slutta        22
Heidi Bruvik Sæther                      Slutta         2
Eirin Nordal                             Ekstern        2
Henriette N. Thommessen                  Ekstern        2
Siam Florelius                           Ekstern        2
Jørgen Kvalsvik                          Slutta        22
Thea Storøy Elnan                        Slutta        63
Knut Hermundstad Aukrust                 Slutta         2
Morten Bakke Kristoffersen               Ekstern        2
Kaia Marie Rosseland                     Ekstern       13
Jon Olsen                                Ekstern        2
Peter Tryggestad                         Slutta        39
Marius Prytz  

Somia Salaouatchi                        Ukjent         1
Line Merete Karlsøen                     Ekstern        1
Truls Oftedal Ellingsen                  Ekstern        1
Rannveig Van Iterson                     Ekstern        1
Jakub Pawlak                             Ekstern        1
Caroline Hovland Svensen                 Ekstern        1
Ellen Dalen                              Ekstern        1
Kaisa R. Hautala                         Ekstern        1
Kunnskapsminister Torbjørn Røe Isaksen   Ekstern        2
Rahma Søvik                              Ukjent         1
Fredrik Scholze                          Slutta        22
Berit Rognhaug                           Ekstern        1
Terje Mørland                            Ekstern        1
Sissel Rogne                             Ekstern        1
Finn-Eirik Johansen                      Ekstern        1
Peter Davidsen                           Slutta         3
Jon Mikkel Broch Ålvik                   Ekstern        1
Anna Nekrosova

Håkon Benjaminsen                        Aktiv         61
Aksel Schreiner Brakestad                Slutta         9
Julianne B. Mossing                      Slutta         6
Kristina Klakegg                         Ekstern        2
Adrian Simen Holm                        Slutta        13
Vida Sundseth Brenna                     Slutta        12
. Translated By Michael Brøndbo          Slutta         4
Birte Magnusson                          Ukjent         1
Jo Helge Ansnes Schei                    Ekstern        1
Joakim P. Berg                           Slutta        21
Erika Kristine Ribu                      Slutta        10
Brian Olguin                             Slutta       340
Sigurd Oland Nedrelid                    Slutta        15
Eline Anker                              Slutta         2
Karl Peder Mork                          Ekstern        1
Magnar Grønvik Müller                    Ekstern        2
Trond Andresen                           Ekstern        1
Pia Ottilia Da

In [25]:
for person in unknowns:
    print(person)
    for stint in person.stint_set.all():
        print(stint)
    for byline in person.byline_set.all():
        print(byline, byline.story.publication_date, byline.story.section)

Emil Øversveen
@bl: Tekst: Emil Øversveen 2015-11-01 18:39:00+00:00 Anmeldelser
Stine Marie Korsfur
@bl: Tekst: Stine Marie Korsfur 2015-11-05 21:19:00+00:00 Nyheter
pia sandved berg
@bl: Tekst: pia sandved berg, Review editor 2016-03-14 10:16:13.412246+00:00 Anmeldelser
Se–Man
@bl: Av: Se–Man 2000-12-26 23:17:00+00:00 Baksiden
@bl: Av: Se–Man 2000-12-26 23:17:00+00:00 Baksiden
Lady Stephanie Huerta Hernandez
@bl: Foto: Lady Stephanie Huerta Hernandez 2015-12-02 14:28:00+00:00 Nyheter
Vilde Boberg Svineng
@bl: Tekst: Vilde Boberg Svineng 2016-02-22 13:06:05.787635+00:00 Anmeldelser
Journalist Malin Kvande
@bl: Tekst: Journalist Malin Kvande 2016-03-28 21:50:00+00:00 Nyheter
Caroline Ulv Johansson
@bl: Tekst: Caroline Ulv Johansson 2016-05-19 12:46:50.200222+00:00 Kultur
Øyunn Syrstad Høydal
@bl: Tekst: Øyunn Syrstad Høydal 2000-12-26 23:17:00+00:00 Kultur
Trygve Bernhardt Moen Haaland
@bl: Av: Trygve Bernhardt Moen Haaland, Formand, Det Norske Studentersamfund 2016-09-12 07:41:00+00:00

@bl: Av: Lars Gule, Studieleder, Høyskolen i Oslo 2007-03-06 23:17:00+00:00 Debatt
@bl: Av: Lars Gule, Postdoktor stipendiat, Høyskolen i Oslo 2011-01-18 23:17:00+00:00 Debatt
Aurora B
@bl: Av: Aurora B 2002-11-19 23:17:00+00:00 Magasin
Too Much Football Woman
@bl: Av: Too Much Football Woman 2002-11-05 23:17:00+00:00 Magasin
F. Nonsens
@bl: Av: F. Nonsens 2003-01-14 23:17:00+00:00 Magasin
Åsmund Øvstehus & Annette Orre
@bl: Tekst: Åsmund Øvstehus & Annette Orre 2003-02-11 23:17:00+00:00 Nyheter
Eivind Yggeseth
@bl: Foto: Eivind Yggeseth 2003-02-18 23:17:00+00:00 Kultur
Anders Aasbø
@bl: Foto: Anders Aasbø 2003-02-18 23:17:00+00:00 Kultur
Dr. Nibbitz
@bl: Av: Dr. Nibbitz 2004-02-17 23:17:00+00:00 Magasin
@bl: Av: Dr. Nibbitz 2004-08-10 23:17:00+00:00 Magasin
@bl: Av: Dr. Nibbitz 2004-11-09 23:17:00+00:00 Magasin
Matrosen
@bl: Av: Matrosen 2004-02-24 23:17:00+00:00 Magasin
Mr. Gjøn
@bl: Av: Mr. Gjøn 2004-03-09 23:17:00+00:00 Magasin
@bl: Av: Mr. Gjøn 2004-11-09 23:17:00+00:00 Magasin
Da

@bl: Av: Aldrende 2006-10-03 23:17:00+00:00 Magasin
Jenny Staurset
@bl: Av: Jenny Staurset 2006-10-03 23:17:00+00:00 Magasin
Audere Est Facere
@bl: Av: Audere Est Facere 2006-10-10 23:17:00+00:00 Magasin
Bukken Von Havresekk
@bl: Av: Bukken Von Havresekk 2006-10-16 23:17:00+00:00 Magasin
Herr Feil
@bl: Av: Herr Feil 2006-10-24 23:17:00+00:00 Magasin
Samselius
@bl: Av: Samselius 2006-10-31 23:17:00+00:00 Magasin
Niklas Lello
@bl: Foto: Niklas Lello 2006-10-31 23:17:00+00:00 Anmeldelser
«Johs. Brun»
@bl: Av: «Johs. Brun» 2006-10-31 23:17:00+00:00 Magasin
Clare Thornton
@bl: Av: Clare Thornton 2006-10-31 23:17:00+00:00 Magasin
Eirin Kongshaug
@bl: Av: Eirin Kongshaug 2006-10-31 23:17:00+00:00 Magasin
@bl: Av: Eirin Kongshaug 2008-02-06 23:17:00+00:00 Magasin
@bl: Av: Eirin Kongshaug, (sminke) 2008-05-27 23:17:00+00:00 Magasin
Nestegangknekkerjegfingradine
@bl: Av: Nestegangknekkerjegfingradine 2006-11-07 23:17:00+00:00 Magasin
Stentrollet Fra Västergötland
@bl: Av: Stentrollet Fra Västerg

@bl: Av: Hanne Prøis Kristiansen 2009-03-31 23:17:00+00:00 Magasin
Gaute Juveth
@bl: Foto: Gaute Juveth 2009-04-21 23:17:00+00:00 Nyheter
Kjell Erik Søvde
@bl: Illustrasjon: Kjell Erik Søvde 2009-04-28 23:17:00+00:00 Feature
Kamilla Simonnes Therese Bongard
@bl: Foto: Kamilla Simonnes Therese Bongard 2009-05-05 23:17:00+00:00 Magasin
For Lån Av Lokaler
@bl: Tekst: For Lån Av Lokaler 2009-05-05 23:17:00+00:00 Magasin
Therese Bongard
@bl: Foto: Therese Bongard 2009-05-05 23:17:00+00:00 Nyheter
Serena Chiara
@bl: Av: Serena Chiara 2009-05-12 23:17:00+00:00 Magasin
Fjodor Sobotka
@bl: Av: Fjodor Sobotka 2009-05-19 23:17:00+00:00 Magasin
Paal Audestad
@bl: Foto: Paal Audestad 2009-08-11 23:17:00+00:00 Feature
Juan Antonio
@bl: Av: Juan Antonio 2009-08-18 23:17:00+00:00 Magasin
Bjørn Ulvang
@bl: Av: Bjørn Ulvang 2009-02-17 23:17:00+00:00 Magasin
Hjernedød
@bl: Av: Hjernedød 2009-10-06 23:17:00+00:00 Magasin
John Lennon
@bl: Av: John Lennon 2009-11-17 23:17:00+00:00 Magasin
Erebos
@bl: Av: Er

@bl: Tekst: Tone Vesterhus, leder av Velferdstinget 2013-10-09 23:17:00+00:00 Nyheter
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-01-15 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-02-26 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-01-15 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-02-26 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, Leder av Velferdstinget i Oslo og Akershus 2013-05-21 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, Leder av Velferdstinget i Oslo og Akershus 2013-05-21 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, Leder av Velferdstinget i Oslo og Akerhus 2013-08-13 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder av Velferdstinget i Oslo og Akershus 2013-09-03 23:17:00+00:00 Debatt
@bl: Av: Tone Vesterhus, leder for Velferdstinget i Oslo og Akershus 2013-11-12 23:17:00+00:00 Debatt
@bl: 

In [72]:
# Fjern ubrukte artikkeltyper

for st in StoryType.objects.all():
    if st.story_set.count() == 0:
        print(st)
        st.delete()
        
for sec in Section.objects.all():
    story_types = sec.storytype_set.all()
    if story_types.count() == 0:
        sec.delete()
    print(sec)
    for story_type in story_types:
        print('   ', story_type, story_type.story_set.count())

Nyheter
    Nett 884
    News 700
    Utenriks 451
    Nyhet 4565
Magasin
    Camillas Kollektion 29
    Reisebilag 18
    Velkomstbilag 62
    Magasin 963
    Petit 273
Kultur
    Akademia 21
    Kultur 2105
    Kulturnotis 2
Debatt
    SMS 192
    Debatt 38
    Leserbrev 2198
    Kronikk 488
Baksiden
    Ad Notam 288
    Vi spør 322
Side 2
    Øyeblikket 125
    Kommentar 572
    Leder 507
Anmeldelser
    Hva skjer 167
    Anmeldelse 3279
Feature
    Reportasje 20
    Studietid 955
    Min studietid 342
Annet
    Universitas Melder 23
    Tassen 9
    Quiz 5
